In [22]:
%%javascript

window.scroll_flag = true
window.scroll_exit = false
window.scroll_delay = 100

$(".output_scroll").each(function() {
    $(this)[0].scrollTop = $(this)[0].scrollHeight;
});

function callScrollToBottom() {
    setTimeout(scrollToBottom, window.scroll_delay);
}

function scrollToBottom() {
    if (window.scroll_exit) {
        return;
    }
    if (!window.scroll_flag) {
        callScrollToBottom();
        return;
    };
    
    $(".output_scroll").each(function() {
        if (!$(this).attr('scroll_checkbox')){
            window.scroll_flag = true;
            $(this).attr('scroll_checkbox',true);
            var div = document.createElement('div');
            var checkbox = document.createElement('input');
            checkbox.type = "checkbox";
            checkbox.onclick = function(){window.scroll_flag = checkbox.checked}
            checkbox.checked = "checked"
            div.append("Auto-Scroll-To-Bottom: ");
            div.append(checkbox);
            $(this).parent().before(div);
        }
        
        $(this)[0].scrollTop = $(this)[0].scrollHeight;
    });
    callScrollToBottom();
}
scrollToBottom();

<IPython.core.display.Javascript object>

In [33]:
import pandas as pd
import numpy as np
import math
import cv2
import dlib
from imutils import face_utils
import imutils
from scipy.spatial import distance as dist
import winsound
import torch
import torch.nn as nn
import torch.optim as optim

In [24]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [25]:
Para = pd.read_csv('Parameters.csv', header = None)
mean = Para[0].to_numpy()
std = Para[1].to_numpy()

In [26]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(6, 10)
        self.fc2 = nn.Linear(10, 64)
        self.fc3 = nn.Linear(64, 128)
        self.fc4 = nn.Linear(128, 256)
        self.fc5 = nn.Linear(256, 512)
        self.fc6 = nn.Linear(512, 256)
        self.fc7 = nn.Linear(256, 128)
        self.fc8 = nn.Linear(128, 64)
        self.fc9 = nn.Linear(64, 32)
        self.fc10 = nn.Linear(32, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = torch.relu(self.fc4(x))
        x = torch.relu(self.fc5(x))
        x = torch.relu(self.fc6(x))
        x = torch.relu(self.fc7(x))
        x = torch.relu(self.fc8(x))
        x = torch.relu(self.fc9(x))
        x = torch.sigmoid(self.fc10(x))
        return x

In [27]:
net = Net()
net.load_state_dict(torch.load('model_weights_1_6.pth'))

<All keys matched successfully>

In [28]:
def EAR(eye):
    A = abs(dist.euclidean(eye[1], eye[5]))
    B = abs(dist.euclidean(eye[2], eye[4]))
    C = abs(dist.euclidean(eye[0], eye[3]))
    ear = (A+B)/(2*C)
    return ear

def MAR(mouth):
    A1=dist.euclidean(mouth[14],mouth[18])
    A2=dist.euclidean(mouth[13],mouth[19])
    A3=dist.euclidean(mouth[15],mouth[17])
    A=(A1+A2+A3)/3.0
    
    B1=dist.euclidean(mouth[1],mouth[5])
    B2=dist.euclidean(mouth[11],mouth[7])
    B3=dist.euclidean(mouth[0],mouth[6]) 
    B=(B1+B2+B3)/3.0
    
    res= A/B
    return res
def MOE(EAR, MAR):
    return MAR / EAR
def PUC(eye):
    A = dist.euclidean(eye[1], eye[4])
    B = dist.euclidean(eye[2], eye[5])
    radius = (A + B) / 4.0
    perimeter = dist.euclidean(eye[0], eye[1]) + dist.euclidean(eye[1], eye[2]) + dist.euclidean(eye[2], eye[3]) + dist.euclidean(eye[3], eye[4]) + dist.euclidean(eye[4], eye[5]) + dist.euclidean(eye[5], eye[0])
    area = math.pi * (radius ** 2)
    circularity = (4 * math.pi * area) / (perimeter ** 2)
    return circularity

In [29]:
ptp= "68 face landmarks.dat" # i have uploded the dat file , it was necessary  ⊙⁠﹏⁠⊙
face_detector = dlib.get_frontal_face_detector() #it will be used just to detect the face 
landmark_detector = dlib.shape_predictor(ptp) # it will be used to landmark points on detected 
(L_start, L_end) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
(R_start, R_end) = face_utils.FACIAL_LANDMARKS_IDXS['right_eye']
(M_start,M_end)=face_utils.FACIAL_LANDMARKS_IDXS['mouth']

In [30]:
temp_initialise = 0.0
alert_threshold = 0.65
new_frame_weightage = 0.6
temp_refresh_rate = 10
mar_threshold = 0.35

In [34]:
take_frame = False
cam = cv2.VideoCapture(0)
temp = temp_initialise
frames_not_drowsy = 0
while 1:
    if (frames_not_drowsy == temp_refresh_rate):
        frames_not_drowsy = 0
        temp = 0
    #checking the frame , if it reaches the end , start again
    if cam.get(cv2.CAP_PROP_POS_FRAMES) == cam.get(cv2.CAP_PROP_FRAME_COUNT) :
        cam.set(cv2.CAP_PROP_POS_FRAMES, 0)
        
    _ , frame = cam.read()
    frame = cv2.resize(frame , (720 , 640))
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_detector(gray, 1)
    if faces.size != 1:
        continue
    for face in faces :
        shape = landmark_detector(gray, face) #applies landmark on all faces avialbe <forms objects>
        shape = face_utils.shape_to_np(shape) # convert all objects created into np.array
        print(type(shape[0]))
        #i.e. it forms corrdinates for 68 landmarks (objects to np.array)
        leftEye = shape[L_start:L_end]
        rightEye = shape[R_start:R_end]
        mouth=shape[M_start:M_end]
        leftEAR = EAR(leftEye)
        rightEAR = EAR(rightEye)
        avg_EAR = (leftEAR + rightEAR)/2.0
        left_circ = PUC(leftEye)
        right_circ = PUC(rightEye)
        Mar=MAR(mouth)
        moe = MOE(avg_EAR, Mar)
        if Mar < mar_threshold :
            Mar = 0
        else :
            Mar = 1
        arr = np.array([leftEAR, rightEAR, Mar, moe, left_circ,right_circ])
        arr = arr-mean
        arr = arr/std
        arr = np.reshape(arr, (1, -1))
        arr = torch.tensor(arr, dtype=torch.float32)
        net.eval()  # set the model to evaluation mode
        with torch.no_grad():
            output = net(arr)
            temp = temp * (1 - new_frame_weightage) + output.item() * new_frame_weightage
            print(temp)
            if (temp < alert_threshold):
                frames_not_drowsy += 1
                take_frame = False
            else:
                frames_not_drowsy = 0
                if (take_frame == True):
                    print("ALERT!!!")
                    winsound.Beep(440, 500)
                else:
                    take_frame = True
    cv2.imshow("Video" ,  gray)
    if cv2.waitKey(3) & 0xff == ord('q'):
        break
cam.release()
cv2.destroyAllWindows

<class 'numpy.ndarray'>
0.25686163902282716
<class 'numpy.ndarray'>
0.28867136120796205
<class 'numpy.ndarray'>
0.2820314176082611
<class 'numpy.ndarray'>
0.2873433712005615
<class 'numpy.ndarray'>
0.4394411874961853
<class 'numpy.ndarray'>
0.38827745251464846
<class 'numpy.ndarray'>
0.4645221874420166
<class 'numpy.ndarray'>
0.48909036101119996
<class 'numpy.ndarray'>
0.3665787072230072
<class 'numpy.ndarray'>
0.37204819088175656
<class 'numpy.ndarray'>
0.1737962007522583
<class 'numpy.ndarray'>
0.2540864324569702
<class 'numpy.ndarray'>
0.3546738495826721
<class 'numpy.ndarray'>
0.44640265188217165
<class 'numpy.ndarray'>
0.3929591468238831
<class 'numpy.ndarray'>
0.38641328776931766
<class 'numpy.ndarray'>
0.42264695630493165
<class 'numpy.ndarray'>
0.5226558274667359
<class 'numpy.ndarray'>
0.5534115126387634
<class 'numpy.ndarray'>
0.393472694979364
<class 'numpy.ndarray'>
0.19060134887695312
<class 'numpy.ndarray'>
0.29493663311004636
<class 'numpy.ndarray'>
0.36370153069496153
<

<function destroyAllWindows>